In [88]:
import flywheel
import pandas as pd
from datetime import datetime
fw = flywheel.Client()
gear = fw.lookup('gears/wscore-ct-heatmap/0.2.1_0.2.0')
gear2 = fw.lookup('gears/antsct-aging-fw')
collection_id = '6184053063bd671113926cc7'
project_id = '5c508d5fc2a4ad002d7628d8' # NACC
collection = fw.get(collection_id)
project = fw.get(project_id)

In [89]:
sessions = [fw.get_session(x.id) for x in fw.get_collection_sessions(collection_id)]

In [98]:
df = pd.read_csv("../PVS_subjects.csv")

## Check what's on Flywheel

In [103]:
# Create a data dict:
data_dict = {'subject':[],'session':[],'wscore_label':[],'run': [], 'status':[], 'ants_label': []}

# Iterate over sessions
for session in sessions:

    # Because we want information off the sessions's analyses, we need to reload
    # The container to make sure we have all the metadata.
    session = session.reload()
    sub_label = session.subject.label
    ses_label = session.label
    # Any analyses on this session will be stored as a list:
    analyses = session.analyses

    # If there are no analyses containers, we know that this gear was not run
    if len(analyses) == 0:
        run = 'False'
        status = 'NA'
        label = 'NA'
        ants_label = 'NA'
    else:
        # Loop through the analyses
        matches = [asys for asys in analyses if asys.gear_info.get('name') == gear.gear.name]
        matches2 = [asys for asys in analyses if asys.gear_info.get('name') == gear2.gear.name]
        
        # check the most recent ants run
        last_ants_date = max([asys.created for asys in matches2]) # test
        last_ants_analysis = [asys for asys in matches2 if asys.created == last_ants_date]
        if len(last_ants_analysis) > 0:
            ants_label = last_ants_analysis[0].label
        else:
            "None"
        
        # If there are no matches, the gear didn't run
        if len(matches) == 0:
            run = 'False'
            status = 'NA'
            wscore_label = 'NA'
        # If there is one match, that's our target
        elif len(matches) == 1:
            # update values 
            run = 'True'
            status = matches[0].job.get('state')
            wscore_label = matches[0].label
            
        # If there are more than one matches (due to reruns), take the most recent run.
        # This behavior may be modified to whatever suits your needs
        else:
            last_wscore_date = max([asys.created for asys in matches])
            last_wscore_analysis = [asys for asys in matches if asys.created == last_wscore_date]
            # There should only be one exact match
            last_wscore_analysis = last_wscore_analysis[0]
            
            # update values 
            run = 'True'
            status = last_wscore_analysis.job.get('state')
            wscore_label = last_wscore_analysis.label

    # Populate our data dict - remember that each key in the data dict must be updated
    # So that the length of our lists stays the same
    data_dict['subject'].append(sub_label)
    data_dict['session'].append(ses_label)
    data_dict['wscore_label'].append(wscore_label)
    data_dict['run'].append(run)
    data_dict['status'].append(status)
    data_dict['ants_label'].append(ants_label)

# Now create a data frame
report_df = pd.DataFrame.from_dict(data_dict)
report_df.head()

,subject,session,wscore_label,run,status,ants_label
0,105476x01,105476x01x20190306x3T,wscore_2021-11-7_23:16_WT_PVS,True,complete,antsct-aging_2021-11-5_17:48_WT_PVS
1,123572,123572x20190313x3T,wscore_2021-05-26_WT,True,complete,antsct_2021-05-13_WT_PVS
2,119493,119493x20190320x3T,wscore_2021-11-7_23:16_WT_PVS,True,complete,antsct-aging_2021-11-5_17:48_WT_PVS
3,123894,123894x20190327x3T,wscore_2021-05-26_WT,True,complete,antsct_2021-05-13_WT_PVS
4,123725,123725x20190327x3T,wscore_2021-05-26_WT,True,complete,antsct_2021-05-13_WT_PVS


In [104]:
# Append a timestamp to our csv name so it won't overwrite anything when we upload it to flywheel
x = datetime.now()
time_str = '%s_%s_%s-%s_%s_%s' % (x.year, x.month, x.day, x.hour, x.minute, x.second) # for the report filename 

csv_out = f'../run_reports/{gear.gear.name}_RunReport_{time_str}.csv'

report_df.to_csv(csv_out,index=False)

collection.upload_file(csv_out)

## Run neuroprint 

In [100]:
# date for label
x = datetime.now()
datestr = '%s-%s-%s_%s:%s' % (x.year, x.month, x.day, x.hour, x.minute)

# get sessions that haven't been analyzed yet
to_run = list(report_df.loc[report_df.run == 'False']['session']) # list of session label strings
for ses_label in to_run:
    ses = project.sessions.find(f"label={ses_label}")[0]
    ses = fw.get(ses.id)
    ants_label = report_df.loc[report_df.session==ses_label]['ants_label'] # get the most recent ants run from the spreadsheet
    if ses.analyses:
        ants_analyses = [asys for asys in ses.analyses if asys.label in str(ants_label)]
        if len(ants_analyses) > 0:
            ants_analysis = ants_analyses[0]
        else:
            continue
        if ants_analysis and ants_analysis.job['state']=='complete' and ants_analysis.files:
            # get antsct zip file
            zip_file = [f for f in ants_analysis.files if f.name.endswith('.zip')][0]
            print('   ' + zip_file.name)
            # get patient age
            age_years = int(df.loc[df.SessionLabel==ses_label]['AgeatMRI'])

            # Run the gear
            asys_label = f'wscore_{datestr}_WT_PVS'
            inputs = {"antsct_output": zip_file}
            config = {"patient_age": age_years, "wthresholds": "0.0 0.5 1.0"}
            print(f'Running neuroprint on {ses_label}...')
            #analysis_id = gear.run(analysis_label=asys_label, config=config, inputs=inputs, destination=ses)


   antsct_sub-105476x01_ses-105476x01x20190306x3T.zip
Running neuroprint on 105476x01x20190306x3T...
   antsct_sub-119493_ses-119493x20190320x3T.zip
Running neuroprint on 119493x20190320x3T...
   antsct_sub-123434_ses-123434x20190402x3T.zip
Running neuroprint on 123434x20190402x3T...
   antsct_sub-112574_ses-112574x20190403x3T.zip
Running neuroprint on 112574x20190403x3T...
   antsct_sub-123950_ses-123950x20190403x3T.zip
Running neuroprint on 123950x20190403x3T...
   antsct_sub-111275x01_ses-111275x01x20190416x3T.zip
Running neuroprint on 111275x01x20190416x3T...
   antsct_sub-104115_ses-104115x20190507x3T.zip
Running neuroprint on 104115x20190507x3T...
   antsct_sub-123072_ses-123072x20190508x3T.zip
Running neuroprint on 123072x20190508x3T...
   antsct_sub-113066x02_ses-113066x02x20190528x3T.zip
Running neuroprint on 113066x02x20190528x3T...
   antsct_sub-100113_ses-100113x20180821x3T.zip
Running neuroprint on 100113x20180821x3T...
   antsct_sub-103941_ses-103941x20180619x3T.zip
Runni